In [1]:
import numpy as np
import itertools

In [2]:
%load_ext autoreload
%autoreload 1

In [3]:
%aimport utils
%aimport classify

In [4]:
classes, class_labels, ffts = utils.read_features(feature='fft')

In [5]:
import sklearn.metrics  as metrics

In [6]:
classes

{'blues': 6,
 'classical': 5,
 'country': 7,
 'disco': 1,
 'hiphop': 0,
 'jazz': 4,
 'metal': 2,
 'pop': 3,
 'reggae': 9,
 'rock': 8}

In [7]:
len(class_labels)


1000

In [8]:
## estimate priors?

class_priors = np.zeros(shape=len(classes))

In [9]:
for k, v in classes.items():
    view = class_labels[class_labels == v]
    class_priors[v] = view.size/class_labels.size

In [10]:
class_priors

array([ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1])

In [11]:
phat = np.zeros((len(classes), ffts.shape[1]))
phat.shape

(10, 1001)

In [12]:
name = 'carly'
chars = set(name)
chars

{'a', 'c', 'l', 'r', 'y'}

In [13]:
ords = (ord(c) for c in set(name))

In [14]:
## I need a big delta matrix for $\delta(y^l = y_j)$

Delta = classify.make_Delta(class_labels, len(classes))



P(Y|W,X)∼exp(WXT), a t×m matrix of probability values. To follow the format of equations (27) and (28) in the text, fill the last row with all 1's, and then normalize each column to sum to one by dividing the each value in the column by the sum of the column.

In [15]:
Delta

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  1.,  0.,  0.]])

Wt+1=Wt+η((Δ−P(Y|W,X))X−λWt)

In [218]:
def retrain(params):
    W = 0 + np.zeros(shape=(len(classes), ffts.shape[1]), dtype='float64')
    W.T[0] = 1
    return classify.grad_decent(ffts, W, Delta, 
                                _lambda=0.1, 
                                eta = 0.001, 
                                rounds = 1000, 
                                debug=True)
W = retrain(None)

Step 0: Error: 0.000000 
 	 updating learning rate: 0.001000
Step 1: Error: 16.039059 
 	 updating learning rate: 0.000999
Step 2: Error: 25.127216 
 	 updating learning rate: 0.000997
Step 3: Error: 31.933625 
 	 updating learning rate: 0.000997
Step 44: Error: 9.650824 
 	 updating learning rate: 0.000995
Step 45: Error: 9.679603 
 	 updating learning rate: 0.000995
Step 46: Error: 9.705486 
 	 updating learning rate: 0.000994
Step 47: Error: 9.728671 
 	 updating learning rate: 0.000992
Step 48: Error: 9.749267 
 	 updating learning rate: 0.000992
Step 49: Error: 9.767385 
 	 updating learning rate: 0.000991
Step 50: Error: 9.783183 
 	 updating learning rate: 0.000989
Step 51: Error: 9.796758 
 	 updating learning rate: 0.000989
Step 52: Error: 9.808205 
 	 updating learning rate: 0.000988
Step 53: Error: 9.817654 
 	 updating learning rate: 0.000986
Step 54: Error: 9.825186 
 	 updating learning rate: 0.000986
Step 55: Error: 9.830890 
 	 updating learning rate: 0.000985
Step 56: 

In [219]:
pred_labs, pred_probs = classify.prediction(W, ffts)

In [220]:
print(metrics.classification_report(class_labels, pred_labs))
print(metrics.confusion_matrix(class_labels, pred_labs))

             precision    recall  f1-score   support

          0       0.82      0.33      0.47       100
          1       0.80      0.52      0.63       100
          2       0.54      0.88      0.67       100
          3       0.71      0.78      0.74       100
          4       0.54      0.31      0.39       100
          5       0.47      0.35      0.40       100
          6       0.52      0.52      0.52       100
          7       0.58      0.41      0.48       100
          8       0.71      0.54      0.61       100
          9       0.21      0.51      0.30       100

avg / total       0.59      0.52      0.52      1000

[[33  1 17  8  4  5  5  1  2 24]
 [ 0 52 13  6  2  9  4  4  2  8]
 [ 0  2 88  0  0  1  4  2  0  3]
 [ 1  1  1 78  3  1  2  3  0 10]
 [ 0  2  4  4 31 10 11  1  1 36]
 [ 0  0  0  0  4 35  3  0  1 57]
 [ 2  1  7  0  6  1 52  8  4 19]
 [ 0  0  7  3  3  5  9 41  7 25]
 [ 1  1 14  5  2  1  5  5 54 12]
 [ 3  5 11  6  2  6  5  6  5 51]]


In [203]:
W

array([[ 0.44525245,  0.03214968,  0.27793166, ...,  0.26749144,
         1.37037199,  0.27793166],
       [ 0.52954111,  0.19124817, -0.25460581, ..., -0.11475964,
        -0.50015548, -0.25460581],
       [ 0.21963872, -0.23126608, -0.14176464, ..., -0.25602386,
        -0.45922344, -0.14176464],
       ..., 
       [ 0.70915681, -0.65018334, -0.21431381, ..., -0.27368333,
        -0.33436681, -0.21431381],
       [ 0.34686242, -0.4370956 , -0.32385314, ..., -0.61308818,
        -0.39901464, -0.32385314],
       [ 3.8073951 ,  1.80220385,  0.63540066, ...,  2.03642194,
         0.3769867 ,  0.63540066]])

In [176]:
pred_probs[0]

array([ 0.79679982,  0.79487006,  0.82568751,  0.91620321,  0.54759224,
        0.44419841,  0.63045316,  0.63479935,  0.74911384,  0.59912873])

In [349]:
_delta = 0
d = 0
f = 0


In [246]:
lp = classify.log_prime(0.1)
s = classify.sigmoid(0.1)

print(lp + s)


1.0


In [131]:
W = np.zeros(shape=(len(classes), ffts.shape[1]), dtype='float64')
W.T[0] = 1
P = classify.P(ffts,W)

In [132]:
P.shape

(10, 1000)

In [133]:
P.T[1:15]

array([[ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
       [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
       [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
       [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
       [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
       [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
       [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
       [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
       [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
       [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
       [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
       [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
       [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
       [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1]])

In [23]:
Px = np.zeros(shape=(P.shape))

In [55]:
den = np.sum(np.log(classify.sigmoid(np.dot(W, ffts.T))), axis=0)

In [56]:
den

array([-6.93147181, -6.93147181, -6.93147181, -6.93147181, -6.93147181,
       -6.93147181, -6.93147181, -6.93147181, -6.93147181, -6.93147181,
       -6.93147181, -6.93147181, -6.93147181, -6.93147181, -6.93147181,
       -6.93147181, -6.93147181, -6.93147181, -6.93147181, -6.93147181,
       -6.93147181, -6.93147181, -6.93147181, -6.93147181, -6.93147181,
       -6.93147181, -6.93147181, -6.93147181, -6.93147181, -6.93147181,
       -6.93147181, -6.93147181, -6.93147181, -6.93147181, -6.93147181,
       -6.93147181, -6.93147181, -6.93147181, -6.93147181, -6.93147181,
       -6.93147181, -6.93147181, -6.93147181, -6.93147181, -6.93147181,
       -6.93147181, -6.93147181, -6.93147181, -6.93147181, -6.93147181,
       -6.93147181, -6.93147181, -6.93147181, -6.93147181, -6.93147181,
       -6.93147181, -6.93147181, -6.93147181, -6.93147181, -6.93147181,
       -6.93147181, -6.93147181, -6.93147181, -6.93147181, -6.93147181,
       -6.93147181, -6.93147181, -6.93147181, -6.93147181, -6.93

In [68]:
classify._logistic(1)

0.7310585786300049

In [27]:
from sklearn.preprocessing import normalize


In [65]:
P = np.exp(np.dot(W, ffts.T)) / den
P = classify.sigmoid(P)


In [67]:
P.T[4]


array([ 0.46399505,  0.46399505,  0.46399505,  0.46399505,  0.46399505,
        0.46399505,  0.46399505,  0.46399505,  0.46399505,  0.46399505])

In [317]:
def log_likelihood(X, Y, w, C=0.1):
    return np.sum(np.log(classify.sigmoid(Y * np.dot(X, w)))) - C/2 * np.dot(w, w)

In [318]:
log_likelihood(ffts, Delta, W)

ValueError: operands could not be broadcast together with shapes (10,1000) (1000,10) 

In [36]:
from scipy.stats import zscore

In [39]:
zscore(P)

array([[-0.33333333,  0.33333333, -0.33333333, ..., -0.33333333,
         0.33333333, -0.33333333],
       [-0.33333333,  0.33333333, -0.33333333, ..., -0.33333333,
         0.33333333, -0.33333333],
       [-0.33333333,  0.33333333, -0.33333333, ..., -0.33333333,
         0.33333333, -0.33333333],
       ..., 
       [-0.33333333,  0.33333333, -0.33333333, ..., -0.33333333,
         0.33333333, -0.33333333],
       [-0.33333333,  0.33333333, -0.33333333, ..., -0.33333333,
         0.33333333, -0.33333333],
       [ 3.        , -3.        ,  3.        , ...,  3.        ,
        -3.        ,  3.        ]])

In [108]:
import random

In [111]:
a = np.arange(9)
a = a.reshape(3,-1)

In [112]:
a

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [113]:
a / np.sum(a, axis=0)

array([[ 0.        ,  0.08333333,  0.13333333],
       [ 0.33333333,  0.33333333,  0.33333333],
       [ 0.66666667,  0.58333333,  0.53333333]])